[https://www.kaggle.com/humananalog/keras-generator-for-reading-directly-from-bson](https://www.kaggle.com/humananalog/keras-generator-for-reading-directly-from-bson)

In [1]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct
import keras

%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import load_img, img_to_array

from collections import defaultdict
from tqdm import *

Using TensorFlow backend.


In [ ]:
np.random.seed(23)

In [2]:
data_dir = "C:\data"

In [3]:
train_bson_path = os.path.join(data_dir, "train.bson")
num_train_products = 7069896

# train_bson_path = os.path.join(data_dir, "train_example.bson")
# num_train_products = 82

test_bson_path = os.path.join(data_dir, "test.bson")
num_test_products = 1768182

## 1. Create lookup tables
The generator uses several lookup tables that describe the layout of the BSON file, which products and images are part of the training/validation sets, and so on.

### 1.1 Lookup table for categories

In [4]:
categories_path = os.path.join(data_dir, "category_names.csv")
categories_df = pd.read_csv(categories_path, index_col="category_id", encoding="mac_latin2")

In [5]:
categories_df.head(5)

,category_level1,category_level2,category_level3
category_id,,,
1000021794,ABONNEMENT / SERVICES,CARTE PREPAYEE,CARTE PREPAYEE MULTIMEDIA
1000012764,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI FUMEUR
1000012776,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI VELO - ABRI MOTO
1000012768,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,FONTAINE A EAU
1000012755,AMENAGEMENT URBAIN - VOIRIE,SIGNALETIQUE,PANNEAU D'INFORMATION EXTERIEUR


In [6]:
# Maps the category_id to an integer index. This is what we'll use to one-hot encode the labels.
categories_df["category_idx"] = pd.Series(range(len(categories_df)), index=categories_df.index)
categories_df.head(5)

,category_level1,category_level2,category_level3,category_idx
category_id,,,,
1000021794,ABONNEMENT / SERVICES,CARTE PREPAYEE,CARTE PREPAYEE MULTIMEDIA,0
1000012764,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI FUMEUR,1
1000012776,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI VELO - ABRI MOTO,2
1000012768,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,FONTAINE A EAU,3
1000012755,AMENAGEMENT URBAIN - VOIRIE,SIGNALETIQUE,PANNEAU D'INFORMATION EXTERIEUR,4


Create dictionaries for quick lookup of `category_id` to `category_idx` mapping.

In [7]:
df = categories_df["category_idx"].reset_index()
cat2idx, idx2cat = dict(zip(df.category_id, df.category_idx)), dict(zip(df.category_idx, df.category_id))

In [8]:
# Test if it works:
cat2idx[1000012755], idx2cat[4] # (4, 1000012755)

(4, 1000012755)

### 1.2 Read the BSON files
We store the offsets and lengths of all items, allowing us random access to the items later.

Inspired by code from: [https://www.kaggle.com/vfdev5/random-item-access]

Note: this takes a few minutes to execute, but we only have to do it once (we'll save the table to a CSV file afterwards).

In [9]:
# We store the offsets and lengths of all items, allowing us random access to the items later.

#num_dicts = 7069896 # according to data page

length_size = 4 # number of bytes decoding item length

def read_bson(bson_path, num_records, with_categories):
    rows = {}
    with open(bson_path, "rb") as f, tqdm(total=num_records) as pbar:
        offset = 0
        while True:
            item_length_bytes = f.read(length_size)
            if len(item_length_bytes) == 0:
                break

            # Decode item length:
            length = struct.unpack("<i", item_length_bytes)[0]

            f.seek(offset)
            item_data = f.read(length)
            assert len(item_data) == length

            item = bson.BSON.decode(item_data)
            product_id = item["_id"]
            num_imgs = len(item["imgs"])

            row = [num_imgs, offset, length]
            if with_categories:
                row += [item["category_id"]]
            rows[product_id] = row

            offset += length
            f.seek(offset)
            pbar.update()

    columns = ["num_imgs", "offset", "length"]
    if with_categories:
        columns += ["category_id"]

    df = pd.DataFrame.from_dict(rows, orient="index")
    df.index.name = "product_id"
    df.columns = columns
    df.sort_index(inplace=True)
    return df

In [10]:
%time train_offsets_df = read_bson(train_bson_path, num_records=num_train_products, with_categories=True)

100%|█████████████████████████████████████████████████████████████████████| 7069896/7069896 [02:48<00:00, 41905.97it/s]


Wall time: 2min 59s


In [11]:
train_offsets_df.head()

,num_imgs,offset,length,category_id
product_id,,,,
0,1,0,6979,1000010653
1,1,6979,7318,1000010653
2,1,14297,5455,1000004079
3,1,19752,4580,1000004141
4,1,24332,6346,1000015539


In [12]:
train_offsets_df.to_csv(os.path.join(data_dir, "train_offsets.csv"))

In [13]:
# How many products?
len(train_offsets_df)

7069896

In [14]:
# How many categories?
len(train_offsets_df["category_id"].unique())

5270

In [15]:
# How many images in total?
train_offsets_df["num_imgs"].sum()

12371293

Also create a table for the offsets from the test set.

In [16]:
%time test_offsets_df = read_bson(test_bson_path, num_records=num_test_products, with_categories=False)

100%|█████████████████████████████████████████████████████████████████████| 1768182/1768182 [00:44<00:00, 39798.90it/s]


Wall time: 47 s


In [17]:
test_offsets_df.head()

,num_imgs,offset,length
product_id,,,
10,3,0,15826
14,1,15826,5589
21,1,21415,7544
24,1,28959,4855
27,1,33814,2921


In [18]:
test_offsets_df.to_csv(os.path.join(data_dir, "test_offsets.csv"))

### 1.3 Create a random train/validation split
We split on products, not on individual images. Since some of the categories only have a few products, we do the split separately for each category.

This creates two new tables, one for the training images and one for the validation images. There is a row for every single image, so if a product has more than one image it occurs more than once in the table.

In [19]:
def make_val_set(train_offsets_df, split_percentage=0.2, drop_percentage=0.):
    # Find the product_ids for each category.
    category_to_products_dict = defaultdict(list)
    for row in tqdm(train_offsets_df.itertuples()):
        category_to_products_dict[row[4]].append(row[0])

    train_list = []
    val_list = []
    with tqdm(total=len(train_offsets_df)) as pbar:
        for category_id, product_ids in category_to_products_dict.items():
            category_idx = cat2idx[category_id]

            # Randomly remove products to make the dataset smaller.
            keep_size = int(len(product_ids) * (1. - drop_percentage))
            if keep_size < len(product_ids):
                product_ids = np.random.choice(product_ids, keep_size, replace=False)

            # Randomly choose the products that become part of the validation set.
            val_size = int(len(product_ids) * split_percentage)
            if val_size > 0:
                val_ids = np.random.choice(product_ids, val_size, replace=False)
            else:
                val_ids = []

            # Create a new row for each image.
            for product_id in product_ids:
                row = [product_id, category_idx]
                for img_idx in range(train_offsets_df.loc[product_id, "num_imgs"]):
                    if product_id in val_ids:
                        val_list.append(row + [img_idx])
                    else:
                        train_list.append(row + [img_idx])
                pbar.update()
                
    columns = ["product_id", "category_idx", "img_idx"]
    train_df = pd.DataFrame(train_list, columns=columns)
    val_df = pd.DataFrame(val_list, columns=columns)   
    return train_df, val_df

Create a 80/20 split. Also drop 90% of all products to make the dataset more manageable. (Note: if drop_percentage > 0, the progress bar doesn't go all the way.)

In [20]:
train_images_df, val_images_df = make_val_set(train_offsets_df, split_percentage=0.25, drop_percentage=0.)

7069896it [00:09, 758936.08it/s]
100%|█████████████████████████████████████████████████████████████████████| 7069896/7069896 [05:24<00:00, 21801.77it/s]


In [21]:
train_images_df.head()

,product_id,category_idx,img_idx
0,42537,619,0
1,55264,619,0
2,104035,619,0
3,114584,619,0
4,123420,619,0


In [22]:
val_images_df.head()

,product_id,category_idx,img_idx
0,36254,619,0
1,121156,619,0
2,121156,619,1
3,140062,619,0
4,224398,619,0


In [23]:
print("Number of training images:", len(train_images_df))
print("Number of validation images:", len(val_images_df))
print("Total images:", len(train_images_df) + len(val_images_df))

Number of training images: 9282339
Number of validation images: 3088954
Total images: 12371293


Are all categories represented in the train/val split? (Note: if the drop percentage is high, then very small categories won't have enough products left to make it into the validation set.)

In [24]:
len(train_images_df["category_idx"].unique()), len(val_images_df["category_idx"].unique())

(5270, 5270)

Quickly verify that the split really is approximately 80-20:

In [25]:
category_idx = 619
num_train = np.sum(train_images_df["category_idx"] == category_idx)
num_val = np.sum(val_images_df["category_idx"] == category_idx)
num_val / num_train

0.32434052757793763

Close enough. ;-) Remember that we split on products but not all products have the same number of images, which is where the slightly discrepancy comes from. (Also, there tend to be fewer validation images if drop_percentage > 0.)

Save the lookup tables as CSV so that we don't need to repeat the above procedure again.

In [26]:
train_images_df.to_csv(os.path.join(data_dir, "train_images.csv"))
val_images_df.to_csv(os.path.join(data_dir, "val_images.csv"))

### 1.4 Lookup table for test set images

Create a list containing a row for each image. If a product has more than one image, it appears more than once in this list.

In [27]:
def make_test_set(test_offsets_df):
    test_list = []
    for row in tqdm(test_offsets_df.itertuples()):
        product_id = row[0]
        num_imgs = row[1]
        for img_idx in range(num_imgs):
            test_list.append([product_id, img_idx])

    columns = ["product_id", "img_idx"]
    test_df = pd.DataFrame(test_list, columns=columns)
    return test_df

In [28]:
test_images_df = make_test_set(test_offsets_df)

1768182it [00:03, 462781.12it/s]


In [29]:
test_images_df.head()

,product_id,img_idx
0,10,0
1,10,1
2,10,2
3,14,0
4,21,0


In [30]:
print("Number of test images:", len(test_images_df))

Number of test images: 3095080


In [31]:
test_images_df.to_csv(os.path.join(data_dir, "test_images.csv"))